![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# Datos semiestrucrados con MongoDB y Apache Spark
## Configuración del ambiente en Google Colaboratory

In [ ]:
# Download Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Next, we will install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# Now, we just need to unzip that folder.
!tar xf spark-3.3.2-bin-hadoop3.tgz

# Setting JVM and Spark path variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# Installing required packages
!pip install -q pyspark==3.3.2
!pip install -q findspark
!pip install -q pymongo

In [ ]:
%%bash
# Install MongoDB
sudo apt install -y mongodb > log
service mongodb start

In [ ]:
import time
# Sleep for few seconds to let the instance start.
time.sleep(5)

In [ ]:
%%bash
# Check running instance of MongoDB
ps -ef | grep mongo

In [ ]:
import re
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

import findspark
findspark.init()

from pyspark import SparkContext

from pymongo import MongoClient

In [ ]:
client = MongoClient()
client.list_database_names() # ['admin', 'local']

In [ ]:
# Download data
!wget https://github.com/words-sdsc/coursera/raw/master/big-data-3/mongodb/dump.tar.gz
!tar xzf dump.tar.gz
!mongorestore dump

In [ ]:
client.list_database_names()

## Acerca de MongoDB
[MongoDB](https://www.mongodb.com) es un sistema de gestión de bases de datos (DBMS) de código abierto y no relacional que utiliza documentos flexibles en lugar de tablas y filas para procesar y almacenar varias formas de datos. Como solución de base de datos NoSQL, MongoDB proporciona un modelo de almacenamiento de datos elástico que permite a los usuarios almacenar y consultar tipos de datos multivariables con facilidad. Esto no solo simplifica la gestión de bases de datos para los desarrolladores, sino que también crea un entorno altamente escalable para aplicaciones y servicios multiplataforma.

Los documentos MongoDB o las colecciones de documentos son las unidades básicas de datos. Formateados como JSON binario (BSON), estos documentos pueden almacenar varios tipos de datos y distribuirse en múltiples sistemas. Dado que MongoDB emplea un diseño de esquema dinámico, los usuarios tienen una flexibilidad sin precedentes a la hora de crear registros de datos, consultar colecciones de documentos a través de la agregación de MongoDB y analizar grandes cantidades de información.

![MongDB doc example](https://webimages.mongodb.com/_com_assets/cms/ldt00m23aaucjidsd-cmab-46.svg)

**Casos de uso**

*Aplicaciones móviles:*
El modelo de documento JSON de MongoDB le permite almacenar datos de aplicaciones de back-end donde los necesite, incluso en dispositivos Apple iOS y Android, así como en soluciones de almacenamiento basadas en la nube. Esta flexibilidad le permite agregar datos a través de múltiples entornos con indexación secundaria y geoespacial, lo que brinda a los desarrolladores la capacidad de escalar sus aplicaciones móviles sin problemas.

*Análisis en tiempo real:*
A medida que las empresas escalan sus operaciones, es fundamental obtener acceso a métricas clave y conocimientos empresariales de grandes conjuntos de datos. MongoDB maneja la conversión de documentos JSON y similares a JSON, como BSON, en objetos Java sin esfuerzo, lo que hace que la lectura y escritura de datos en MongoDB sea rápida e increíblemente eficiente al analizar información en tiempo real en múltiples entornos de desarrollo. Esto ha demostrado ser beneficioso para varios sectores empresariales, incluidos el gobierno, los servicios financieros y el comercio minorista.

*Sistemas de gestión de contenidos:*
Los sistemas de gestión de contenidos (CMS) son herramientas poderosas que desempeñan un papel importante para garantizar experiencias de usuario positivas al acceder a sitios de comercio electrónico, publicaciones en línea, plataformas de gestión de documentos y otras aplicaciones y servicios. Al usar MongoDB, puede agregar fácilmente nuevas características y atributos a sus aplicaciones y sitios web en línea utilizando una sola base de datos y con alta disponibilidad.

*Almacén de datos empresariales:*
El marco Apache Hadoop es una colección de módulos de código abierto, incluidos Hadoop Distributed File System y Hadoop MapReduce, que trabajan con MongoDB para almacenar, procesar y analizar grandes cantidades de datos. Las organizaciones pueden usar MongoDB y Hadoop para realizar modelos de riesgos, análisis predictivos y procesamiento de datos en tiempo real.

Información tomada en parte de [IBM](https://www.ibm.com/topics/mongodb)

## Funciones básicas de MongoDB
Para empezar podemos seleccionar la base de datos con la que queremos trabajar usando la misma sintaxis de diccionarios, y podemos visualizar las colecciones o documentos en una base de datos con `list_collection_names`.

In [ ]:
db = client['sample']
db.list_collection_names()

Podemos usar el método `count_documents` para ver cuántos registros tenemos en nuestra colección..

In [ ]:
db.users.count_documents({})

Para examinar el contenido de uno de los registros, podemos usar el método `find_one`.

In [ ]:
db.users.find_one()

Aquí podemos notar que cada registro tiene distintos campos o llaves, por ejemplo, *user_name*, *retweet_count*, *tweet_ID*, etc., y campos anidados bajo *user*,por ejemplo, *CreatedAt*, *UserId*, *Location*, etc.

Para encontrar los valores únicos en un campo específico podemos usar el método `distinct`, por ejemplo, los siguientes son los valores únicos para el campo *user_name*:

In [ ]:
db.users.distinct('user_name')

Las *queries* o consultas se realizan con el método `find` donde el primer parámetro son las condiciones de búsqueda. 

In [ ]:
db.users.find({'user_name':'Hobbision123'})

El cual retorna un objeto *Cursor* sobre el que podemos iterar para mostrar los resultados.

In [ ]:
from pprint import pprint # Para darle formato a los resultados y hacerlos más legibles
for registro in db.users.find({'user_name':'Hobbision123'}):
  pprint(registro)

El segundo parámetro son los atributos que deseo obtener de la consulta.

In [ ]:
for registro in db.users.find({'user_name':'Hobbision123'}, {'tweet_text':True}):
  pprint(registro)

In [ ]:
for registro in db.users.find({'user_name':'Hobbision123'}, {'tweet_text':True, '_id':False}):
  pprint(registro)

Si utilizamos el método `count_documents` con una condición de búsqueda obtenemos el número de registros que cumplen la condición.

In [ ]:
db.users.count_documents({'user_name':'Hobbision123'})

Las consultas se hacen por comparación literal o textual, si realizamos la búsqueda por la palabra *FIFA* en el contenido del tweet no vamos a obtener ningún resultado, pues no hay ningún tweet que sea solo esa palabra.

In [ ]:
db.users.count_documents({'tweet_text':'FIFA'})

Por el contrario, si queremos encontrar los tweets en los que aparece la palabra *FIFA*, debemos usar expresiones regulares (*regEx*) para buscar ese patrón en el texto de los tweets

In [ ]:
import re
regex = re.compile(r'FIFA')
db.users.count_documents({'tweet_text':regex})

In [ ]:
# Con el argumento "limit" indicamos cuántos resultados queremos retornar
for registro in db.users.find({'tweet_text':regex}, limit=2):
  pprint(registro)

De manera similar para valores númericos, la siguiente expresión solo retorna los registros en los que el campo *tweet_mentioned_count* tiene un valor de 6.

In [ ]:
db.users.count_documents({"tweet_mentioned_count":6})

Para otro tipo de comparaciones debemos utilizar operadores propios de **MongoDB** como `$gt` para comparaciones mayor que (greater than).

Una lista completa de los operadores se puede encontrar en la [documentación](https://www.mongodb.com/docs/drivers/node/current/fundamentals/crud/query-document/).

In [ ]:
db.users.count_documents({"tweet_mentioned_count":{'$gt':6}})

In [ ]:
for registro in db.users.find({"tweet_mentioned_count":{'$gt':6}}):
  pprint(registro)

Para consultas con comparaciones entre campos del mismo registro, se debe utilizar el operador `$expr` como en el siguiente ejemplo:

In [ ]:
db.users.count_documents({'$expr':
                          {'$gt':['$tweet_mentioned_count', '$tweet_followers_count']}
                          })

In [ ]:
for registro in db.users.find({'$expr':
                               {'$gt':['$tweet_mentioned_count', '$tweet_followers_count']}
                               }
                              ,limit=1):
  pprint(registro)

Para consultas con varias condiciones se usan los operadores lógicos como se muestra a continuación:

In [ ]:
db.users.count_documents({'$and':[{"tweet_mentioned_count":{'$gt':4}}, {'tweet_text':regex}]})

In [ ]:
for registro in db.users.find({'$and':[{"tweet_mentioned_count":{'$gt':4}}, {'tweet_text':regex}]}):
  pprint(registro)

## Ejercicios

Basándose en los ejemplos vistos en este notebook y en la [documentación](https://www.mongodb.com/docs/drivers/node/current/fundamentals/crud/query-document/), resuelva:

**1.** ¿Cuántos registros cumplen con $tweet\_mentioned\_count < 4$?

**2.** ¿Cuántos registros cumplen con $tweet\_mentioned\_count > 4$ o $tweet\_followers\_count \leq 4$? Muestre 5.

**3.** Extraiga todos los textos de los tweets en la colección y realice el conteo de palabras utilizando la función creada con Apache Spark en la sesión anterior.

**¿Qué operaciones adicionales de limpieza de texto considera necesarias?**